# Download a data file from the UCL server

We're using a small [example data set](http://data.cortexlab.net/singlePhase3/) recorded by [Nick Steinmetz](http://www.nicksteinmetz.com/) at UCL.

In [1]:
import urllib.request
file_url = 'http://data.cortexlab.net/singlePhase3/data/rawDataSample.bin'
file_name = "rawDataSample.bin"

urllib.request.urlretrieve(file_url, file_name)

('rawDataSample.bin', <http.client.HTTPMessage at 0x7f993c373860>)

# Use of the `HSDetection` class

In [1]:
import sys
sys.path.append('..')
from hs2 import HSDetection
from probe import NeuroPixel
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
%matplotlib inline

/home/cole/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
# raw data location (as downloaded above)
data_path = file_name


# detection parameters
to_localize = True
cutout_start = 6
cutout_end = 30
threshold = 22
file_directory = 'results/'
file_name = 'ProcessedSpikesNeuropixel'

In [8]:
Probe = NeuroPixel(data_path)
H = HSDetection(Probe, to_localize, cutout_start, cutout_end, threshold,
                maa=0, maxsl=12, minsl=3, ahpthr=0, out_file_name=file_name, 
                file_directory_name=file_directory, save_all=True)

In [9]:
# Probe.show([0])

In [10]:
H.DetectFromRaw()

# Sampling rate: 30000
# Localization On
# Not Masking any Channels
# Writing out extended detection info
# Number of recorded channels: 385
# Analysing frames: 1800000, Seconds:60.0
# Frames before spike in cutout: 30
# Frames after spike in cutout: 66
# tcuts: 42 55
# tInc: 50000
# Analysing 50000 frames; from -42 to 50055
# Analysing 50000 frames; from 49958 to 100055
# Analysing 50000 frames; from 99958 to 150055
# Analysing 50000 frames; from 149958 to 200055
# Analysing 50000 frames; from 199958 to 250055
# Analysing 50000 frames; from 249958 to 300055
# Analysing 50000 frames; from 299958 to 350055
# Analysing 50000 frames; from 349958 to 400055
# Analysing 50000 frames; from 399958 to 450055
# Analysing 50000 frames; from 449958 to 500055
# Analysing 50000 frames; from 499958 to 550055
# Analysing 50000 frames; from 549958 to 600055
# Analysing 50000 frames; from 599958 to 650055
# Analysing 50000 frames; from 649958 to 700055
# Analysing 50000 frames; from 699958 to 750055
# A

In [7]:
H.LoadDetected()

Detected and read 113268 spikes.


In [ ]:
plt.figure(figsize=(10, 10))
H.PlotTracesChannels(2000, window_size=100)

In [ ]:
plt.figure(figsize=(14, 3))
ax = plt.subplot(111)
H.PlotAll(invert=True, s=1, alpha=0.2, ax=ax)
ax.set_xlim((0,500))

# Use of the `HSClustering` class

In [ ]:
from hs2 import HSClustering
# Load from file
#C = Clustering(['results/ProcessedSpikes_mea1k.bin', 'results/ProcessedSpikes_mea1k.bin'], cutout_length=41)

# Or if the spikes are already in memory
# simply load from the Detection class
C = HSClustering(H)

In [ ]:
%%time
C.ShapePCA(pca_ncomponents=2, pca_whiten=True);

In [ ]:
%%time
C.CombinedClustering(alpha=5, bandwidth=5., bin_seeding=False, n_jobs=-1,cluster_subset=50000)

In [ ]:
plt.figure(figsize=(14, 4))
ax = plt.subplot(111)
C.PlotAll(invert=True, s=1, ax=ax, max_show=100000,show_labels=False)
plt.title("MeanShift, bandwidth=.3, no bin seeding");
ax.set_xlim((0,500))


In [ ]:
units = range(16)
C.PlotShapes(units)

In [ ]:
C.PlotNeighbourhood(1,radius=2, alpha=0.8);

In [ ]:
# save results:
C.SaveHDF5(file_name+"_sorted.hdf5")